In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
titanik = sns.load_dataset('titanic')
titanik

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [3]:
titanik['embarked'].value_counts(),titanik['embark_town'].value_counts()

(embarked
 S    644
 C    168
 Q     77
 Name: count, dtype: int64,
 embark_town
 Southampton    644
 Cherbourg      168
 Queenstown      77
 Name: count, dtype: int64)

In [4]:
titanik['survived'].value_counts(),titanik['alive'].value_counts()

(survived
 0    549
 1    342
 Name: count, dtype: int64,
 alive
 no     549
 yes    342
 Name: count, dtype: int64)

In [5]:
titanik['sex'].value_counts(),titanik['who'].value_counts()

(sex
 male      577
 female    314
 Name: count, dtype: int64,
 who
 man      537
 woman    271
 child     83
 Name: count, dtype: int64)

In [6]:
titanik['pclass'].value_counts(),titanik['class'].value_counts()

(pclass
 3    491
 1    216
 2    184
 Name: count, dtype: int64,
 class
 Third     491
 First     216
 Second    184
 Name: count, dtype: int64)

In [7]:
titanik.drop(['embarked','alive','sex','pclass'], axis = 1, inplace=True)

In [8]:
titanik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   age          714 non-null    float64 
 2   sibsp        891 non-null    int64   
 3   parch        891 non-null    int64   
 4   fare         891 non-null    float64 
 5   class        891 non-null    category
 6   who          891 non-null    object  
 7   adult_male   891 non-null    bool    
 8   deck         203 non-null    category
 9   embark_town  889 non-null    object  
 10  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(2)
memory usage: 52.8+ KB


In [9]:
titanik.describe()

,survived,age,sibsp,parch,fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,29.699118,0.523008,0.381594,32.204208
std,0.486592,14.526497,1.102743,0.806057,49.693429
min,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
titanik.head(10)

,survived,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alone
0,0,22.0,1,0,7.2500,Third,man,True,NaN,Southampton,False
1,1,38.0,1,0,71.2833,First,woman,False,C,Cherbourg,False
2,1,26.0,0,0,7.9250,Third,woman,False,NaN,Southampton,True
3,1,35.0,1,0,53.1000,First,woman,False,C,Southampton,False
4,0,35.0,0,0,8.0500,Third,man,True,NaN,Southampton,True
5,0,NaN,0,0,8.4583,Third,man,True,NaN,Queenstown,True
6,0,54.0,0,0,51.8625,First,man,True,E,Southampton,True
7,0,2.0,3,1,21.0750,Third,child,False,NaN,Southampton,False
8,1,27.0,0,2,11.1333,Third,woman,False,NaN,Southampton,False
9,1,14.0,1,0,30.0708,Second,child,False,NaN,Cherbourg,False


In [11]:
titanik.columns

Index(['survived', 'age', 'sibsp', 'parch', 'fare', 'class', 'who',
       'adult_male', 'deck', 'embark_town', 'alone'],
      dtype='object')

In [12]:
titanik['class'].value_counts()

class
Third     491
First     216
Second    184
Name: count, dtype: int64

In [13]:
titanik['deck'].value_counts()

deck
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: count, dtype: int64

In [14]:
titanik.groupby('deck', observed=True)['age'].mean()

deck
A    44.833333
B    34.955556
C    36.086667
D    39.032258
E    38.116667
F    19.954545
G    14.750000
Name: age, dtype: float64

In [15]:
pd.crosstab(titanik['deck'], titanik['class'])

class,First,Second,Third
deck,,,
A,15,0,0
B,47,0,0
C,59,0,0
D,29,4,0
E,25,4,3
F,0,8,5
G,0,0,4


In [16]:
pd.crosstab(titanik['deck'], titanik['who'])

who,child,man,woman
deck,,,
A,1,13,1
B,3,19,25
C,2,31,26
D,0,15,18
E,1,16,15
F,4,5,4
G,2,0,2


In [17]:
x = titanik[['deck','who','class','age','fare']]
x

,deck,who,class,age,fare
0,NaN,man,Third,22.0,7.2500
1,C,woman,First,38.0,71.2833
2,NaN,woman,Third,26.0,7.9250
3,C,woman,First,35.0,53.1000
4,NaN,man,Third,35.0,8.0500
...,...,...,...,...,...
886,NaN,man,Second,27.0,13.0000
887,B,woman,First,19.0,30.0000
888,NaN,woman,Third,NaN,23.4500
889,C,man,First,26.0,30.0000


In [18]:
y = titanik[['deck']]
y.head(15)

,deck
0,NaN
1,C
2,NaN
3,C
4,NaN
5,NaN
6,E
7,NaN
8,NaN
9,NaN


In [19]:
deckEnc = OrdinalEncoder()
y = deckEnc.fit_transform(y)

In [20]:
y

array([[nan],
       [ 2.],
       [nan],
       [ 2.],
       [nan],
       [nan],
       [ 4.],
       [nan],
       [nan],
       [nan],
       [ 6.],
       [ 2.],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [ 3.],
       [nan],
       [ 0.],
       [nan],
       [nan],
       [nan],
       [ 2.],
       [nan],
       [nan],
       [nan],
       [ 1.],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [ 3.],
       [nan],
       [ 1.],
       [ 2.],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [ 1.],
       [ 2.],
       [nan],
       [nan],
       [nan],
       [ 5.],
       [nan],
       [nan],
       [nan],
       [nan],
      

In [21]:
ordinal = x[['deck', 'class']]
ordinal

,deck,class
0,NaN,Third
1,C,First
2,NaN,Third
3,C,First
4,NaN,Third
...,...,...
886,NaN,Second
887,B,First
888,NaN,Third
889,C,First


In [22]:
dummy = x[['who']]
dummy

,who
0,man
1,woman
2,woman
3,woman
4,man
...,...
886,man
887,woman
888,woman
889,man


In [23]:
numeric = x[['age','fare']]
numeric

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
...,...,...
886,27.0,13.0000
887,19.0,30.0000
888,NaN,23.4500
889,26.0,30.0000


In [24]:
x

,deck,who,class,age,fare
0,NaN,man,Third,22.0,7.2500
1,C,woman,First,38.0,71.2833
2,NaN,woman,Third,26.0,7.9250
3,C,woman,First,35.0,53.1000
4,NaN,man,Third,35.0,8.0500
...,...,...,...,...,...
886,NaN,man,Second,27.0,13.0000
887,B,woman,First,19.0,30.0000
888,NaN,woman,Third,NaN,23.4500
889,C,man,First,26.0,30.0000


In [25]:
ordinalTrans = OrdinalEncoder()
dummyTrans = OneHotEncoder( handle_unknown= 'ignore')
numericTrans = StandardScaler()

In [26]:
preprocessor = ColumnTransformer([('ordinal', ordinalTrans,  ['deck', 'class']),
                                 ('dummy', dummyTrans, ['who']),
                                 ('numeric', numericTrans, ['age','fare'])])
# preprocessor = ColumnTransformer([
#     ('ordinal', ordinalTrans, ['deck', 'class']),
#     ('dummy', dummyTrans, ['who']),
#     ('numeric', numericTrans, ['age', 'fare'])
# ])

imputer = KNNImputer()
pipe = Pipeline([('preproc', preprocessor),
                 ('impute', imputer)])

In [27]:
pipe.fit_transform(x)

array([[ 4.4       ,  2.        ,  0.        , ...,  0.        ,
        -0.53037664, -0.50244517],
       [ 2.        ,  0.        ,  0.        , ...,  1.        ,
         0.57183099,  0.78684529],
       [ 5.        ,  2.        ,  0.        , ...,  1.        ,
        -0.25482473, -0.48885426],
       ...,
       [ 4.8       ,  2.        ,  0.        , ...,  1.        ,
         0.2411687 , -0.17626324],
       [ 2.        ,  0.        ,  0.        , ...,  0.        ,
        -0.25482473, -0.04438104],
       [ 4.8       ,  2.        ,  0.        , ...,  0.        ,
         0.15850313, -0.49237783]])

In [28]:
xtrans = pipe.fit_transform(x)

In [29]:
ordinal = pipe.named_steps['preproc'].named_transformers_['ordinal'].inverse_transform(xtrans[:, :2])
dummy = pipe.named_steps['preproc'].named_transformers_['dummy'].inverse_transform(xtrans[:, 2:5])
numeric = pipe.named_steps['preproc'].named_transformers_['numeric'].inverse_transform(xtrans[:, -2:])

In [30]:
print(ordinal[:5], dummy[:5], numeric[:5], sep ='\n')

[['E' 'Third']
 ['C' 'First']
 ['F' 'Third']
 ['C' 'First']
 ['E' 'Third']]
[['man']
 ['woman']
 ['woman']
 ['woman']
 ['man']]
[[22.      7.25  ]
 [38.     71.2833]
 [26.      7.925 ]
 [35.     53.1   ]
 [35.      8.05  ]]


In [31]:
x['deck'].value_counts()

deck
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: count, dtype: int64

In [32]:
x1 = pd.concat([
        pd.DataFrame(ordinal, columns=['deck', 'class']),
        pd.DataFrame(dummy, columns=['who']),
        pd.DataFrame(numeric, columns=['age', 'fare'])
    ], axis=1)
x1

,deck,class,who,age,fare
0,E,Third,man,22.0,7.2500
1,C,First,woman,38.0,71.2833
2,F,Third,woman,26.0,7.9250
3,C,First,woman,35.0,53.1000
4,E,Third,man,35.0,8.0500
...,...,...,...,...,...
886,D,Second,man,27.0,13.0000
887,B,First,woman,19.0,30.0000
888,E,Third,woman,33.2,23.4500
889,C,First,man,26.0,30.0000


In [33]:
for i in sorted(x1['deck'].unique()):
    print('Pre\n',titanik[titanik['deck'] == i]['class'].value_counts(),'\n Posle\n',x1[x1['deck'] == i]['class'].value_counts(),'\n\n')

Pre
 class
First     15
Second     0
Third      0
Name: count, dtype: int64 
 Posle
 class
First    16
Name: count, dtype: int64 


Pre
 class
First     47
Second     0
Third      0
Name: count, dtype: int64 
 Posle
 class
First     67
Second     5
Name: count, dtype: int64 


Pre
 class
First     59
Second     0
Third      0
Name: count, dtype: int64 
 Posle
 class
First     75
Second    25
Third      1
Name: count, dtype: int64 


Pre
 class
First     29
Second     4
Third      0
Name: count, dtype: int64 
 Posle
 class
Second    61
First     33
Name: count, dtype: int64 


Pre
 class
First     25
Second     4
Third      3
Name: count, dtype: int64 
 Posle
 class
Third     354
Second     73
First      25
Name: count, dtype: int64 


Pre
 class
Second    8
Third     5
First     0
Name: count, dtype: int64 
 Posle
 class
Third     132
Second     20
Name: count, dtype: int64 


Pre
 class
Third     4
First     0
Second    0
Name: count, dtype: int64 
 Posle
 class
Third    4
Name: count

In [34]:
titanik[x1.columns] = x1

In [69]:
titanik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   age          891 non-null    float64
 2   sibsp        891 non-null    int64  
 3   parch        891 non-null    int64  
 4   fare         891 non-null    float64
 5   class        891 non-null    object 
 6   who          891 non-null    object 
 7   adult_male   891 non-null    bool   
 8   deck         891 non-null    object 
 9   embark_town  889 non-null    object 
 10  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(3), object(4)
memory usage: 64.5+ KB
